In [92]:
import keras
from keras.utils import plot_model
from keras.models import load_model
from metrics import dice_coefficient,dice_coef,dice_coefficient_loss,dice_coef_loss,weighted_dice_coefficient,weighted_dice_coefficient_loss
import numpy as np
from keras.layers import Input,Conv3D,Flatten,Dense,Model,Add,MaxPooling3D
import pydot
print(pydot.__version__)

1.4.1


In [111]:
def load_old_model(model_file):
    print("Loading pre-trained model")
    custom_objects = {'dice_coefficient_loss': dice_coefficient_loss, 'dice_coefficient': dice_coefficient,
                      'dice_coef': dice_coef, 'dice_coef_loss': dice_coef_loss,
                      'weighted_dice_coefficient': weighted_dice_coefficient,
                      'weighted_dice_coefficient_loss': weighted_dice_coefficient_loss}
    try:
        from keras_contrib.layers import InstanceNormalization
        custom_objects["InstanceNormalization"] = InstanceNormalization
    except ImportError:
        pass
    try:
        return load_model(model_file, custom_objects=custom_objects)
    except ValueError as error:
        if 'InstanceNormalization' in str(error):
            raise ValueError(str(error) + "\n\nPlease install keras-contrib to use InstanceNormalization:\n"
                                          "'pip install git+https://www.github.com/keras-team/keras-contrib.git'")
        else:
            raise error
            

In [112]:
model = load_old_model('3dunet.h5')

Loading pre-trained model


In [113]:
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 4, 64, 64, 64 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 32, 64, 64, 6 3488        input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 64, 64, 6 0           conv3d_1[0][0]                   
__________________________________________________________________________________________________
conv3d_2 (Conv3D)               (None, 64, 64, 64, 6 55360       activation_1[0][0]               
____________________________________________________________________________________________

In [117]:
startModel = keras.layers.Sequential([Conv3D(filters = 4,kernel_size = (1,1,1),input_shape= (2,128,128,128),data_format='channels_first')
                                      ,MaxPooling3D(pool_size=(2,2,2),data_format='channels_first')])

startModel.summary()

test_tensor = np.random.rand(1,2,128,128,128)
print(startModel.predict(test_tensor).shape)



Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_19 (Conv3D)           (None, 4, 128, 128, 128)  12        
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 4, 64, 64, 64)     0         
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________
Tensor("max_pooling3d_16/transpose_1:0", shape=(?, 4, 64, 64, 64), dtype=float32)
(1, 4, 64, 64, 64)


In [116]:
 # pops the input layer
mergedMod = Model(startModel.input,model.output)

ValueError: Graph disconnected: cannot obtain value for tensor Tensor("input_1_9:0", shape=(?, 4, 64, 64, 64), dtype=float32) at layer "input_1". The following previous layers were accessed without issue: []

In [99]:
startModel.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_10 (Conv3D)           (None, 4, 128, 128, 128)  12        
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 4, 64, 64, 64)     0         
_________________________________________________________________
input_1 (InputLayer)         (None, 4, 64, 64, 64)     0         
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________


In [110]:
model.summary()





Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
activation_1 (Activation)       (None, 32, 64, 64, 6 0           conv3d_1[0][0]                   
__________________________________________________________________________________________________
conv3d_2 (Conv3D)               (None, 64, 64, 64, 6 55360       activation_1[0][0]               
__________________________________________________________________________________________________
activation_2 (Activation)       (None, 64, 64, 64, 6 0           conv3d_2[0][0]                   
__________________________________________________________________________________________________
max_pooling3d_1 (MaxPooling3D)  (None, 64, 32, 32, 3 0           activation_2[0][0]               
____________________________________________________________________________________________

In [ ]:
from keras.layers import Input,Model
newinput = Input(batch_shape=(None,4,10,64,64)) # 5D as the first parameter is the size of the batch
newOutput = model(newinput)
newModel = Model(newinput,newOutput)

newModel.summary()

In [5]:
# NISSEBRESH JÄVLA ARTIKEL

#https://stackoverflow.com/questions/42187425/how-to-change-input-shape-in-sequential-model-in-keras
import keras
import numpy as np

keras.backend.clear_session()

  
def change_model(model, new_input_shape=(None, 40, 40, 3)):
    # replace input shape of first layer
    model._layers[0].batch_input_shape = new_input_shape

    # rebuild model architecture by exporting and importing via json
    new_model = keras.models.model_from_json(model.to_json())

    # copy weights from old model to new one
    for layer in new_model.layers:
        try:
            layer.set_weights(model.get_layer(name=layer.name).get_weights())
            print("Loaded layer {}".format(layer.name))
        except:
            print("Could not transfer weights for layer {}".format(layer.name))

    return new_model
print('hej')
new_model = change_model(model,new_input_shape=(1,4,64,64,64))

hej


KeyError: 'input_1_ib-0'

<function pydot.call_graphviz(program, arguments, working_dir, **kwargs)>